In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [6]:
# Load the data
file_path = Path('./resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [7]:
# PREPROCESSING

# Convert target to binary within column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df2 = df.copy()
df2['loan_status'] = le.fit_transform(df2['loan_status'])

# Convert date to integer

# Months dictionary
months_num = {
    "Jan-2019": 1,
    "Feb-2019": 2,
    "Mar-2019": 3,
    "Apr-2019": 4,
    "May-2019": 5
 }

# Months' names encoded using the dictionary values
df2["issue_d"] = df2["issue_d"].apply(lambda x: months_num[x])
df2["next_pymnt_d"] = df2["next_pymnt_d"].apply(lambda x: months_num[x])

# Convert columns with string values to numerical ones
df_bin = pd.get_dummies(df2, columns=["application_type","initial_list_status","home_ownership", "verification_status", "pymnt_plan", "hardship_flag", "debt_settlement_flag"])

df_bin.head()

,loan_amnt,int_rate,installment,annual_inc,issue_d,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,3,1,27.24,0.0,0.0,8.0,...,0,0,0,1,0,1,0,1,1,1
1,25000.0,0.2000,929.09,105000.0,3,1,20.23,0.0,0.0,17.0,...,0,1,0,0,0,0,1,1,1,1
2,20000.0,0.2000,529.88,56000.0,3,1,24.26,0.0,0.0,8.0,...,0,1,0,0,0,0,1,1,1,1
3,10000.0,0.1640,353.55,92000.0,3,1,31.44,0.0,1.0,10.0,...,0,0,0,1,0,0,1,1,1,1
4,22000.0,0.1474,520.39,52000.0,3,1,18.76,0.0,1.0,14.0,...,0,1,0,0,1,0,0,1,1,1


# Split the Data into Training and Testing

In [8]:
# Create our features
X = df_bin.drop(columns='loan_status')


# Create our target
y = df_bin['loan_status']

In [9]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,1.726172,21.778153,0.217766,0.497697,12.587340,0.126030,...,0.009285,0.526309,0.106747,0.357659,0.478007,0.373992,0.148001,1.0,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,0.743862,20.199244,0.718367,0.758122,6.022869,0.336797,...,0.095914,0.499311,0.308793,0.479314,0.499520,0.483865,0.355104,0.0,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,1.000000,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,1.000000,13.890000,0.000000,0.000000,8.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,2.000000,19.760000,0.000000,0.000000,11.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,2.000000,26.660000,0.000000,1.000000,16.000000,0.000000,...,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,3.000000,999.000000,18.000000,5.000000,72.000000,4.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0


In [10]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(51612, 92)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [20]:
# Resample the training data with the BalancedRandomForestClassifier

from imblearn.ensemble import BalancedRandomForestClassifier

# Create a random forest classifier.
rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1) 

# Fitting the model
rf_model = rf_model.fit(X_train, y_train)



In [22]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7732866391900439

In [23]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[   57,    30],
       [ 1859, 15259]], dtype=int64)

In [24]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.03      0.66      0.89      0.06      0.76      0.57        87
          1       1.00      0.89      0.66      0.94      0.76      0.60     17118

avg / total       0.99      0.89      0.66      0.94      0.76      0.60     17205



In [25]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.08079385160405511, 'total_rec_prncp'),
 (0.06370238209644011, 'last_pymnt_amnt'),
 (0.0634175770535778, 'total_rec_int'),
 (0.05026154421498834, 'total_pymnt'),
 (0.0479341892466518, 'total_pymnt_inv'),
 (0.025959373336813023, 'int_rate'),
 (0.024918956404946267, 'issue_d'),
 (0.020066870456937218, 'out_prncp'),
 (0.01973909733378538, 'dti'),
 (0.01835984727396918, 'mths_since_recent_inq'),
 (0.01654557478935291, 'mo_sin_old_rev_tl_op'),
 (0.015678665980897247, 'bc_util'),
 (0.015470996004163162, 'mo_sin_old_il_acct'),
 (0.015398323248394399, 'max_bal_bc'),
 (0.015369874297764765, 'all_util'),
 (0.01529695266613003, 'tot_hi_cred_lim'),
 (0.015279026181341662, 'mths_since_rcnt_il'),
 (0.015195204586446133, 'avg_cur_bal'),
 (0.01517520329427647, 'il_util'),
 (0.015085708924139568, 'installment'),
 (0.015046710672389536, 'revol_bal'),
 (0.014956589240691206, 'out_prncp_inv'),
 (0.014331997400623001, 'annual_inc'),
 (0.014253498281212506, 'total_rev_hi_lim'),
 (0.01395340224962224, 'to

### Easy Ensemble AdaBoost Classifier

In [26]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier 

# Create a random forest classifier.
ez_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)

# Fitting the model
ez_model = ez_model.fit(X_train, y_train)


In [28]:
# Calculated the balanced accuracy score
y_pred = ez_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9251060589579028

In [29]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[   79,     8],
       [  990, 16128]], dtype=int64)

In [30]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.07      0.91      0.94      0.14      0.92      0.85        87
          1       1.00      0.94      0.91      0.97      0.92      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.92      0.86     17205

